Problem Statement. <br/>

A city's skyline is the outer contour of the silhouette formed by all the buildings in that city when viewed from a distance. Given the locations and heights of all the buildings, return the skyline formed by these buildings collectively. <br/>
The geometric information of each building is given in the array buildings where buildings[i] = [lefti, righti, heighti]: <br/>

    lefti is the x coordinate of the left edge of the ith building.
    righti is the x coordinate of the right edge of the ith building.
    heighti is the height of the ith building.

You may assume all buildings are perfect rectangles grounded on an absolutely flat surface at height 0. <br/>

The skyline should be represented as a list of "key points" sorted by their x-coordinate in the form [[x1,y1],[x2,y2],...]. Each key point is the left endpoint of some horizontal segment in the skyline except the last point in the list, which always has a y-coordinate 0 and is used to mark the skyline's termination where the rightmost building ends. Any ground between the leftmost and rightmost buildings should be part of the skyline's contour. <br/>
Note: There must be no consecutive horizontal lines of equal height in the output skyline. For instance, [...,[2 3],[4 5],[7 5],[11 5],[12 7],...] is not acceptable; the three lines of height 5 should be merged into one in the final output as such: [...,[2 3],[4 5],[12 7],...] <br/>

Example 1: <br/>
Input: buildings = [[2,9,10],[3,7,15],[5,12,12],[15,20,10],[19,24,8]] <br/>
Output: [[2,10],[3,15],[7,12],[12,0],[15,10],[20,8],[24,0]] <br/>
Explanation: <br/>
Figure A shows the buildings of the input. <br/>
Figure B shows the skyline formed by those buildings. The red points in figure B represent the key points in the output list. <br/>

Example 2: <br/>
Input: buildings = [[0,2,3],[2,5,3]] <br/>
Output: [[0,3],[5,0]]

# Heights Array - O(N * X) runtime, O(X) space where X is the largest X co-ordinate and N is the number of buildings

In [1]:
from typing import List
from collections import deque

class Solution:
    def getSkyline(self, buildings: List[List[int]]) -> List[List[int]]:
        if not buildings:
            return [[0, 0]]
        
        n = len(buildings)
        if n == 1:
            return[[buildings[0][0], buildings[0][2]], [buildings[0][1], 0]]
        left = buildings[0][0]
        zeroes = deque()
        heights = []
        result = []
                                
        for building in buildings:
            width = len(heights) + left - 1
            if building[1] > width:
                heights += [0 for _ in range(building[1] - width)]
                if width >= left and building[0] - width == 1:
                    zeroes.append(width)
            for i in range(building[0], building[1] + 1):
                heights[i - left] = max(heights[i - left], building[2])
                
        #print(heights, zeroes)
                
        result.append([left, heights[0]])
        for i in range(1, len(heights)):
            if heights[i] > heights[i - 1]:
                if zeroes and left + i - 1 == zeroes[0]:
                    zeroes.popleft()
                    result.append([left + i, 0])
                    result.append([left + i + 1, heights[i]])
                else:
                    result.append([left + i, heights[i]])
            elif heights[i] < heights[i - 1]:
                if zeroes and left + i - 1 == zeroes[0]:
                    zeroes.popleft()
                    result.append([left + i - 1, 0])
                    result.append([left + i, heights[i]])
                else:
                    result.append([left + i - 1, heights[i]])
        if result and result[-1][1] != 0:
            result.append([left + i, 0])
            
        return result

# Divide and Conquer - O(N log N) runtime, O(N) space,where N is the number of buildings

In [2]:
from typing import List

class Solution:
    def getSkyline(self, buildings: List[List[int]]) -> List[List[int]]:

        n = len(buildings)
        # The base cases
        if n == 0:
            return []
        if n == 1:
            x_start, x_end, y = buildings[0]
            return [[x_start, y], [x_end, 0]]

        # If there is more than one building,
        # recursively divide the input into two subproblems.
        left_skyline = self.getSkyline(buildings[: n // 2])
        right_skyline = self.getSkyline(buildings[n // 2 :])

        # Merge the results of subproblem together.
        return self.merge_skylines(left_skyline, right_skyline)

    def merge_skylines(self, left, right):
        """
        Merge two skylines together.
        """
        def update_output(x, y):
            """
            Update the final output with the new element.
            """
            # if skyline change is not vertical -
            # add the new point
            if not output or output[-1][0] != x:
                output.append([x, y])
            # if skyline change is vertical -
            # update the last point
            else:
                output[-1][1] = y

        def append_skyline(p, lst, n, y, curr_y):
            """
            Append the rest of the skyline elements with indice (p, n)
            to the final output.
            """
            while p < n:
                x, y = lst[p]
                p += 1
                if curr_y != y:
                    update_output(x, y)
                    curr_y = y

        n_l, n_r = len(left), len(right)
        p_l = p_r = 0
        curr_y  = left_y = right_y = 0
        output = []

        # while we're in the region where both skylines are present
        while p_l < n_l and p_r < n_r:
            point_l, point_r = left[p_l], right[p_r]
            # pick up the smallest x
            if point_l[0] < point_r[0]:
                x, left_y = point_l
                p_l += 1
            else:
                x, right_y = point_r
                p_r += 1
            # max height (i.e. y) between both skylines
            max_y = max(left_y, right_y)
            # if there is a skyline change
            if curr_y != max_y:
                update_output(x, max_y)
                curr_y = max_y

        # there is only left skyline
        append_skyline(p_l, left, n_l, left_y, curr_y)

        # there is only right skyline
        append_skyline(p_r, right, n_r, right_y, curr_y)

        return output

# Sort and Heap - O(N log N) runtime, O(N) space,where N is the number of buildings

In [4]:
from typing import List
from heapq import heappush, heappop

class Solution(object):
    def getSkyline(self, buildings: List[List[int]]) -> List[List[int]]:
        # add start-building events
        # also add end-building events(acts as buildings with 0 height)
        # and sort the events in left -> right order
        events = [(L, -H, R) for L, R, H in buildings]
        events += list({(R, 0, 0) for _, R, _ in buildings})
        events.sort()

        # res: result, [x, height]
        # live: heap, [-height, ending position]
        res = [[0, 0]]
        live = [(0, float("inf"))]
        for pos, negH, R in events:
            # 1, pop buildings that are already ended
            # 2, if it's the start-building event, make the building alive
            # 3, if previous keypoint height != current highest height, edit the result
            while live[0][1] <= pos: 
                heappop(live)
            if negH: 
                heappush(live, (negH, R))
            if res[-1][1] != -live[0][0]:
                res += [ [pos, -live[0][0]] ]
        return res[1:]

In [5]:
instance = Solution()
instance.getSkyline([[2,9,10],[3,7,15],[5,12,12],[15,20,10],[19,24,8]])

[[2, 10], [3, 15], [7, 12], [12, 0], [15, 10], [20, 8], [24, 0]]